In [1]:
from unityagents import UnityEnvironment
import numpy as np

import matplotlib.pyplot as plt
import torch

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using device: ", device)

env = UnityEnvironment(file_name="../Tennis_Linux/Tennis.x86_64", no_graphics=False)

brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
num_agents = len(env_info.agents)
action_size = brain.vector_action_space_size
states = env_info.vector_observations
state_size = states.shape[1]

using device:  cuda:0


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [5]:
from maddpg_agents import Maddpg
import torch
from collections import namedtuple, deque
import random
import matplotlib.pyplot as plt
import pandas as pd


In [8]:
maddpg = Maddpg(state_size, action_size, num_agents, 10)

maddpg.agents[0].actor_local.load_state_dict(torch.load('checkpoint_actor_local_0_5808.pth'))



maddpg.agents[1].actor_local.load_state_dict(torch.load('checkpoint_actor_local_1_5808.pth'))


all_steps=0
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    noise = 1
    while True:
        all_steps+=1
        
        actions = maddpg.act(states, noise)                    # retrieve actions to performe for each agents 
        noise *= 0.995                               # Decrease action noise
        env_info = env.step(actions)[brain_name]               # send all actions to tne environment
        next_states = env_info.vector_observations             # get next state for each agent
        rewards = env_info.rewards                             # get reward (for each agent)
        dones = env_info.local_done                            # see if episode finished

        maddpg.step(states, actions, rewards, next_states, dones, all_steps)  
        scores += env_info.rewards                             # update the score (for each agent)
        states = next_states                                   # roll over states to next time step
        if np.any(dones):                                      # exit loop if episode finished
            break

    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

/home/pongsasit/anaconda3/envs/drlnd/lib/python3.6/site-packages/torch/nn/functional.py:1698: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/media/pongsasit/480/deepRlUdacity/deep-reinforcement-learning/p3_collab-compet/tennis2/maddpg_agents.py:131: UserWarning: Using a target size (torch.Size([200, 2])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  critic_loss = F.mse_loss(Q_expected, Q_targets)


Score (max over agents) from episode 1: 0.6900000106543303
Score (max over agents) from episode 2: 0.10000000149011612
Score (max over agents) from episode 3: 0.10000000149011612
Score (max over agents) from episode 4: 0.09000000171363354
Score (max over agents) from episode 5: 0.09000000171363354


In [9]:
env.close()